In [1]:
from johnsnowlabs import *
spark = jsl.start(json_license_path="...")

22/10/24 15:11:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched cpu-Optimized JVM SparkSession with Jars for: 🚀Spark-NLP==4.2.1, 💊Spark-Healthcare==4.2.0, running on ⚡ PySpark==3.1.2


In [2]:
documentAssembler = nlp.DocumentAssembler() \
     .setInputCol("text") \
     .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
     .setInputCols(["document"])\
     .setOutputCol("token")

embeddings = nlp.LongformerEmbeddings.pretrained("legal_longformer_base", "en")\
    .setInputCols("document", "token") \
    .setOutputCol("embeddings")

sembeddings = nlp.SentenceEmbeddings()\
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

consulting = nlp.ClassifierDLModel.pretrained('legclf_consulting_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_consulting_agreement")

credit = nlp.ClassifierDLModel.pretrained('legclf_credit_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_credit_agreement")

employment = nlp.ClassifierDLModel.pretrained('legclf_employment_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_employment_agreement")

lease = nlp.ClassifierDLModel.pretrained('legclf_lease_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_lease_agreement")

loan = nlp.ClassifierDLModel.pretrained('legclf_loan_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_loan_agreement")

management = nlp.ClassifierDLModel.pretrained('legclf_management_contract', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_management_contract")

purchase = nlp.ClassifierDLModel.pretrained('legclf_purchase_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_purchase_agreement")

service = nlp.ClassifierDLModel.pretrained('legclf_service_agreement', 'en', 'legal/models') \
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("is_service_agreement")

legal_longformer_base download started this may take some time.
Approximate size to download 506.5 MB
[ | ]legal_longformer_base download started this may take some time.
Approximate size to download 506.5 MB
[ — ]Download done! Loading the resource.
[ — ]

2022-10-24 15:12:52.074023: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]
legclf_consulting_agreement download started this may take some time.
Approximate size to download 20.3 MB
[ | ]legclf_consulting_agreement download started this may take some time.
Approximate size to download 20.3 MB
Download done! Loading the resource.
[OK!]
legclf_credit_agreement download started this may take some time.
Approximate size to download 20.5 MB
[ | ]legclf_credit_agreement download started this may take some time.
Approximate size to download 20.5 MB
Download done! Loading the resource.
[OK!]
legclf_employment_agreement download started this may take some time.
Approximate size to download 19.8 MB
[ | ]legclf_employment_agreement download started this may take some time.
Approximate size to download 19.8 MB
Download done! Loading the resource.
[OK!]
legclf_lease_agreement download started this may take some time.
Approximate size to download 20 MB
[ | ]legclf_lease_agreement download started this may take some time.
Approximate size to download 20 MB
[ / ]Downlo

In [4]:
pipeline = nlp.Pipeline(stages = [
    documentAssembler,
    tokenizer,
    embeddings,
    sembeddings,
    consulting,
    credit,
    employment,
    lease,
    loan,
    management,
    purchase,
    service])

In [5]:
import os

texts = []

for root, dirs, files in os.walk("./documents"):
    for d in dirs:
        with open(f"./documents/{d}/0.txt", 'r') as f:
            text = f.read()
            texts.append((d, text))
    break

In [6]:
import pandas as pd

df = pd.DataFrame(texts, columns=['class', 'text'])
df

,class,text
0,credit-agreement,Exhibit 10.1SECOND AMENDMENT TO AMENDED AND RE...
1,purchase-agreement,EXHIBIT 99.2\...
2,management-contract,AMENDED and RESTATED MANAGEMENT CONTRACT ...
3,service-agreement,\nEXHIBIT 10.27\n \n \n \nSERVICE AGREEMENT\n...
4,consulting-agreement,"\n 000 Xxxxx Xxxxxx Xxxx Xxxxxxxx, Xxxx 00000\..."
5,lease-agreement,\n\n 1\n ...
6,loan-agreement,\nExhibit 6.7\n \nAMENDED AND RESTATED LOAN AG...
7,employment-agreement,\n\n\n EXHIBI...


In [7]:
categories = df['class'].tolist()
categories

['credit-agreement',
 'purchase-agreement',
 'management-contract',
 'service-agreement',
 'consulting-agreement',
 'lease-agreement',
 'loan-agreement',
 'employment-agreement']

In [8]:
sdf = spark.createDataFrame([[""]]).toDF("text")

In [9]:
fitpipe = pipeline.fit(sdf)
lp = LightPipeline(fitpipe)

In [10]:
for i, t in enumerate(texts):
    res = lp.fullAnnotate(t[1])
    print(f"expected, model, result, positive_confidence, negative_confidence") 
    for x in res[0].keys():    
        if x.startswith("is_"):
            model = x.replace("is_","").replace("_","-")
            yes_no = "FALSE" if res[0][x][0].result == 'other' else "TRUE"
            print(f"{categories[i]}, legclf_{model}, {yes_no}, {res[0][x][0].metadata[model]},{res[0][x][0].metadata['other']}")
    print()

expected, model, result, positive_confidence, negative_confidence
credit-agreement, legclf_employment-agreement, FALSE, 6.256421E-7,0.9999994
credit-agreement, legclf_consulting-agreement, FALSE, 6.9225584E-6,0.9999931
credit-agreement, legclf_purchase-agreement, FALSE, 1.91974E-4,0.999808
credit-agreement, legclf_management-contract, FALSE, 3.867073E-7,0.99999964
credit-agreement, legclf_service-agreement, FALSE, 6.862126E-6,0.9999931
credit-agreement, legclf_credit-agreement, TRUE, 0.9993468,6.5319834E-4
credit-agreement, legclf_loan-agreement, FALSE, 0.21249497,0.787505
credit-agreement, legclf_lease-agreement, FALSE, 0.0015328855,0.998467

expected, model, result, positive_confidence, negative_confidence
purchase-agreement, legclf_employment-agreement, FALSE, 6.3946317E-7,0.9999994
purchase-agreement, legclf_consulting-agreement, FALSE, 3.3757082E-5,0.99996626
purchase-agreement, legclf_purchase-agreement, TRUE, 0.9996309,3.6900994E-4
purchase-agreement, legclf_management-contract,